# Basics of Mobile Robotics - Main Notebook


In [ ]:
# Import libraries
import numpy as np
import cv2 as cv
from tdmclient import ClientAsync
from Local_Nav import Local_Nav as LN
from Global_Nav import Global_Nav as GN
# import RepeatedTimer

# Thymio Parametres

In [ ]:
r = 2 # radius of the wheel (cm)
L = 9.5 # distance from wheel to wheel (cm)
v_max = 20 # cm/s
cruising = 150 
motor_speed_max = 500
omega_to_motor = (motor_speed_max*r)/v_max
pixel_to_cm = 1/6

# Controller Parametres

In [ ]:
Kp = 10   # Proporcional gain
Ki = 0.1  # Integral gain
Kd = 1    # Derivative gain
error_sum = 0       # Integral error
previous_error = 0  # Derivative error

# Kalman Filter Parametres

In [ ]:
ts = 0.1  # time step
A = np.array([[1, ts, 0, 0, 0, 0],  # state transition matrix
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, ts, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, ts],
              [0, 0, 0, 0, 0, 1]])

P0 = GN.covariance(20, 5, 20, 5, 20, 5)  # state covariance matrix

Q = GN.covariance(0.01, 0.1, 0.01, 0.1, 0.001, 0.01)  # process noise covariance matrix

R = GN.covariance(1, 0.5, 1, 0.5, 0.1, 0.01)  # measurement covariance matrix

In [ ]:
client = ClientAsync()
node = client.aw(client.wait_for_node())
client.aw(node.lock_node())

# Define global_nav object with class attributes
global_nav = GN(client, node, cruising, ts, Kp, Ki, Kd, error_sum, previous_error, r, L, omega_to_motor, R, Q, A, P0)


In [ ]:
#---------------------------------------------------------------------------
#                                 Find shortest Path
#---------------------------------------------------------------------------


# Inicial states
X = np.array([      [0],    # x_dot0
        [thymio_pose[0]],    # x0
                     [0],    # y_dot0
        [thymio_pose[1]],    # y0
                     [0],    # phi_dot0
        [thymio_pose[2]]])   # phi0 

path_cm = path*pixel_to_cm
vid = cv.VideoCapture(1+ cv.CAP_DSHOW)  
while(True):
      
    # Capture the video frame
    # by frame
    ret, img = vid.read()
    next_point = 0
    while (np.abs(X[1,0]-path_cm[-1][1]) > 1) and (np.abs(X[3,0]-path_cm[-1][0]) > 1):
        while (np.abs(X[3,0]-path_cm[next_point][1]) > 1) and (np.abs(X[1,0]-path_cm[next_point][0]) > 1):
            node = client.aw(client.wait_for_node())
            # Calculate desired angle
            phi_d = np.arctan((path[next_point][1]-X[3,0])/(path[next_point].x-X[1,0]))
            # Control action to make the Thymio follow the reference
            vr, vl = GN.PIDcontroller(phi_d, X[5,0])
            motor_left = vl*omega_to_motor
            motor_right = vr*omega_to_motor            
            # Saturation
            if motor_left > 500:
                motor_left = 500
            elif motor_right > 500:
                motor_right = 500
            elif motor_right < -500:
                motor_right = -500
            elif motor_left < -500:
                motor_left = -500            
            # Activate Obstacle avoidance if necessary
            # Define local_nav object with class attributes
            local_nav = LN(client, node, motor_left, motor_right)
            flag = local_nav.analyse_data()
            if flag == 1:
                task = local_nav.obstacle_avoidance() #trigger task depending on if flag detected
            else:
                adjust_speed = GN.motors(motor_left, motor_right)
                node.send_set_variables(adjust_speed)
                node.flush()
            # Update states
            motor_sensor = GN.get_sensor_data()
            speed_sensor = motor_sensor/GN.omega_to_motor      # cm/s
            thymio_xy_pix,thymio_pose,_ = GN.find_thymio(img, pixel_to_cm)
            if thymio_pose != 0: # if the camera can find the thymio
                X[1,0] = thymio_pose[0]                                          # x
                X[3,0] = thymio_pose[1]                                          # y
                X[5,0] = thymio_pose[2]                                          # phi
                X[4,0] = (r/L)*(speed_sensor[1]-speed_sensor[0])                 # phi_dot
                X[0,0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.cos(X[5,0])  # x_dot
                X[2,0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.sin(X[5,0])  # y_dot
                # Draw a point on the frame
                img = cv.circle(img, (thymio_xy_pix[0], thymio_xy_pix[1]), radius=10, color=(0, 0, 255), thickness=-1)
            else:
                X[4,0] = (r/L)*(speed_sensor[1]-speed_sensor[0])                 # phi_dot
                X[5,0] += X[4,0]*ts                                              # phi
                X[0,0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.cos(X[5,0])  # x_dot
                X[2,0] = (r/2)*(speed_sensor[0]+speed_sensor[1])*np.sin(X[5,0])  # y_dot
                X[1,0] += X[0,0]*ts                                              # x
                X[3,0] += X[3,0]*ts                                              # y
            X = GN.kalman_filter(X)
            cv.imshow("Video", img)
            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        next_point += 1
        cv.imshow("Video", img)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    motor_left = 0
    motor_right = 0
    adjust_speed = GN.motors(motor_left, motor_right)
    node.send_set_variables(adjust_speed)
    node.flush()
    cv.imshow("Video", img)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv.waitKey(1) & 0xFF == ord('q'):
        break







In [3]:
# from tdmclient import ClientAsync
# import Local_Nav as LN
# import Global_Nav as GN

# client = ClientAsync()
# node = client.aw(client.wait_for_node())

# # Define all necessary variables for what comes after

# # Define global_nav object with class attributes
# global_nav = GN.Global_Nav(client, node, cruising, ts, Kp, Ki, Kd, ...
#                            error_sum, previous_error, r, L, omega_to_motor, R, Q, A, X0, P0)

# # Define local_nav object with class attributes
# local_nav = LN.Local_Nav(client, node, motor_speed_left, motor_speed_right)


SyntaxError: invalid syntax (<ipython-input-3-feacd1b74e29>, line 9)

In [4]:
# #----------------------------------------------------
# #          FINAL MAIN STRUCTURE TRYOUT
# #---------------------------------------------------


# # #----------------------------------------------------
# # Computer Vision part and getting the shortest path
# #---------------------------------------------------
# ''' From there we should obtain a "control sequence" of where and when Thymio should turn. This shall later be compared to
# the current position of the Thymio (obtained by tracking) and we want to align the directional vector with the shortest path'''

# #--------------------
# # Global Navigation
# #--------------------

# # pos_goal = ...
# # path_vector = ... #generate_path_vector

# client.aw(node.lock_node()) #allows to update thymio variables as we loop later on
# colour_switch = local_nav.colour(3) #Make Thymio blue when in standby, because it's cool
# node.flush()

# While pos_goal != |current_thymio_pos - 5| #5cm error ?

#     node = client.aw(client.wait_for_node()) # Update states and sensor values at each iteration
#     flag = local_nav.analyse_data() # store a flag=1 if obstacle detected (red LEDS)
#     global_nav.update_states()
    
#     controlled_speed = global_nav.actuation(next_point)
#     node.send_set_variables(controlled_speed) #
    
#     #--------------------
#     # Local Navigation
#     #--------------------
 
#     if flag == 1:
#         task = local_nav.obstacle_avoidance() #trigger task depending on if flag detected
#     else:
#         pass
    

# node.flush()

SyntaxError: invalid syntax (<ipython-input-4-a2476ea36da0>, line 15)